In [8]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, "../src/")
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from process import Doc2VecProcessor
from vectorize import Doc2VecVectorizer
from utils import get_doc_paths, read_labels
import pathlib

print(os.getcwd())
raw_docs_dir = "../data/raw_docs/"
labels_dir = "../data/labels/"


print(labels_dir)
os.makedirs("../data/processed_sentences/")
os.makedirs("../data/processed_sentences/eng/")
os.makedirs("../data/processed_sentences/ger/")
os.makedirs("../data/processed_doc2vec_sc_500_st_500/")
os.makedirs("../data/processed_doc2vec_sc_500_st_500/eng/")
os.makedirs("../data/processed_doc2vec_sc_500_st_500/ger/")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/annina/scripts/great_unread_nlp/nbs
../data/labels/


# Process data

In [2]:
lang = "eng"
raw_doc_paths = get_doc_paths(raw_docs_dir, lang)
dp = Doc2VecProcessor(lang=lang, processed_chunk_sentence_count=None, stride=None)
dp.process(raw_doc_paths)

INFO:root:Processing texts...
100%|██████████| 599/599 [01:20<00:00,  7.41it/s]
INFO:root:Processed texts.


In [3]:
lang = "ger"
raw_doc_paths = get_doc_paths(raw_docs_dir, lang)
dp = Doc2VecProcessor(lang=lang, processed_chunk_sentence_count=None, stride=None)
dp.process(raw_doc_paths)

INFO:root:Processing texts...
100%|██████████| 547/547 [00:55<00:00,  9.79it/s]
INFO:root:Processed texts.


In [9]:
lang = "eng"
raw_doc_paths = get_doc_paths(raw_docs_dir, lang)
dp = Doc2VecProcessor(lang=lang, processed_chunk_sentence_count=500, stride=500)
dp.process(raw_doc_paths)

INFO:root:Processing texts...
100%|██████████| 599/599 [5:47:55<00:00, 34.85s/it]    
INFO:root:Processed texts.


In [14]:
lang = "ger"
raw_doc_paths = get_doc_paths(raw_docs_dir, lang)
dp = Doc2VecProcessor(lang=lang, processed_chunk_sentence_count=500, stride=500)
dp.process(raw_doc_paths)

INFO:root:Processing texts...
100%|██████████| 547/547 [4:27:49<00:00, 29.38s/it]    
INFO:root:Processed texts.


# Full documents + Doc2VecDMM + SVR

In [20]:
lang = "eng"
processed_full_doc_paths = get_doc_paths("../data/processed_doc2vec_full/", lang)
d2vv = Doc2VecVectorizer(dm=1, dm_mean=1)
d2vv.fit(processed_full_doc_paths)
df = d2vv.get_doc_vectors()

INFO:root:Fitting Doc2VecVectorizer...
INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:collected 176680 word types and 599 unique tags from a corpus of 599 examples and 60607461 words
INFO:gensim.models.word2vec:Creating a fresh vocabulary
DEBUG:gensim.utils:starting a new internal lifecycle event log for Doc2Vec
INFO:gensim.utils:Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 66900 unique words (37.86506678741227%% of original 176680, drops 109780)', 'datetime': '2021-05-13T21:21:14.243380', 'gensim': '4.0.1', 'python': '3.9.4 (default, Apr  9 2021, 16:34:09) \n[GCC 7.3.0]', 'platform': 'Linux-5.8.0-50-generic-x86_64-with-glibc2.31', 'event': 'prepare_vocab'}
INFO:gensim.utils:Doc2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 60438101 word corpus (99.72056245682359%% of original 60607461, drops 169360)', 'datetime': '

DEBUG:gensim.models.word2vec:worker exiting, processed 73 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 4 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 79 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 3 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 71 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 75 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 68 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH - 4 : training on 60607461 raw words (5076946 effective words) took 5.0s, 1018059 effective words/s
INFO:gensim.models.word2vec:EPOCH 5 - PROGRESS: at 21.87% examples, 1109539 words/s, in_qsize 15, out_qsize 0
INFO:gensim

INFO:gensim.models.word2vec:EPOCH 9 - PROGRESS: at 45.58% examples, 1167582 words/s, in_qsize 16, out_qsize 2
INFO:gensim.models.word2vec:EPOCH 9 - PROGRESS: at 69.45% examples, 1190663 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 9 - PROGRESS: at 94.99% examples, 1192147 words/s, in_qsize 15, out_qsize 0
DEBUG:gensim.models.word2vec:job loop exiting, total 585 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 75 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 7 more threads
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 6 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 75 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 73 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 5 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 72 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 4 more threads
DEBUG:gensim

In [ ]:
labels = read_labels(labels_dir, "eng")
df['y'] = df["doc_path"].apply(lambda x: labels[x.split("/")[-1][:-4]])
df = df.drop(columns=['doc_path'])

from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

X = df.drop(columns=['y']).values
y = df["y"].values.ravel()

In [19]:
all_predictions = []
all_labels = []
kf = KFold(n_splits=10, shuffle=True, random_state=42)
for index, (train_indices, validation_indices) in enumerate(kf.split(X)):
    train_X = X[train_indices, :]
    train_y = y[train_indices]
    validation_X = X[validation_indices, :]
    validation_y = y[validation_indices]
    
    model = SVR(C=30)
    model.fit(train_X, train_y)
    train_yhat = model.predict(train_X)
    validation_yhat = model.predict(validation_X)
    all_labels.extend(validation_y.tolist())
    all_predictions.extend(validation_yhat.tolist())
    train_mse = mean_squared_error(train_y, train_yhat)
    train_mae = mean_absolute_error(train_y, train_yhat)
    validation_mse = mean_squared_error(validation_y, validation_yhat)
    validation_mae = mean_absolute_error(validation_y, validation_yhat)
    print(f"Fold: {index+1}, TrainMSE: {train_mse}, TrainMAE: {train_mae}, ValMSE: {validation_mse}, ValMAE: {validation_mae}")
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

import matplotlib.pyplot as plt

plt.figure(figsize=(18, 6))
plt.hist(all_labels)
plt.show();

plt.figure(figsize=(18, 6))
plt.scatter(all_labels, all_predictions)
plt.show();

../src/utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_df["file_name"][labels_df["file_name"] == key] = value


KeyError: 'doc_path'

# Chunks of 500 Sentences + Doc2VecDMM + SVR

In [13]:
lang = "eng"
processed_sc_500_st_500_doc_paths = get_doc_paths("../data/processed_doc2vec_sc_500_st_500/", lang)
d2vv = Doc2VecVectorizer(dm=1, dm_mean=1)
d2vv.fit(processed_sc_500_st_500_doc_paths)
df = d2vv.get_doc_vectors()
labels = read_labels("eng")
df['y'] = df["doc_path"].apply(lambda x: labels[x.split("/")[-1][:-4].split("_pt")[0]])
df['book_name'] = df['doc_path'].apply(lambda x: x.split("/")[-1][:-4].split("_pt")[0])


from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

all_predictions = []
all_labels = []

book_names = df['book_name'].unique()
book_names_splitted = np.array_split(book_names, 10)

for index, split in enumerate(book_names_splitted):
    train_X = df[~df["book_name"].isin(split)].drop(columns=["y", "doc_path", "book_name"]).values
    train_y = df[~df["book_name"].isin(split)]["y"].values.ravel()
    validation_X = df[df["book_name"].isin(split)].drop(columns=["y", "doc_path", "book_name"]).values
    validation_y = df[df["book_name"].isin(split)]["y"].values.ravel()
    
    model = SVR(C=30)
    model.fit(train_X, train_y)
    train_yhat = model.predict(train_X)
    validation_yhat = model.predict(validation_X)
    all_labels.extend(validation_y.tolist())
    all_predictions.extend(validation_yhat.tolist())
    train_mse = mean_squared_error(train_y, train_yhat)
    train_mae = mean_absolute_error(train_y, train_yhat)
    validation_mse = mean_squared_error(validation_y, validation_yhat)
    validation_mae = mean_absolute_error(validation_y, validation_yhat)
    print(f"Fold: {index+1}, TrainMSE: {train_mse}, TrainMAE: {train_mae}, ValMSE: {validation_mse}, ValMAE: {validation_mae}")
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

import matplotlib.pyplot as plt

plt.figure(figsize=(18, 6))
plt.scatter(all_labels, all_predictions)

# Full documents + Doc2VecDBOW + SVR

lang = "eng"
processed_full_doc_paths = get_doc_paths("../data/processed_doc2vec_full/", lang)
d2vv = Doc2VecVectorizer(dm=0, dm_mean=0)
d2vv.fit(processed_full_doc_paths)
df = d2vv.get_doc_vectors()
labels = read_labels("eng")
df['y'] = df["doc_path"].apply(lambda x: labels[x.split("/")[-1][:-4]])
df = df.drop(columns=['doc_path'])

from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

X = df.drop(columns=['y']).values
y = df["y"].values.ravel()

all_predictions = []
all_labels = []
kf = KFold(n_splits=10, shuffle=True, random_state=42)
for index, (train_indices, validation_indices) in enumerate(kf.split(X)):
    train_X = X[train_indices, :]
    train_y = y[train_indices]
    validation_X = X[validation_indices, :]
    validation_y = y[validation_indices]
    
    model = MLPRegressor(hidden_layer_sizes=(80, 50, 30, 10), activation="relu", max_iter=50)
    model.fit(train_X, train_y)
    train_yhat = model.predict(train_X)
    validation_yhat = model.predict(validation_X)
    all_labels.extend(validation_y.tolist())
    all_predictions.extend(validation_yhat.tolist())
    train_mse = mean_squared_error(train_y, train_yhat)
    train_mae = mean_absolute_error(train_y, train_yhat)
    validation_mse = mean_squared_error(validation_y, validation_yhat)
    validation_mae = mean_absolute_error(validation_y, validation_yhat)
    print(f"Fold: {index+1}, TrainMSE: {train_mse}, TrainMAE: {train_mae}, ValMSE: {validation_mse}, ValMAE: {validation_mae}")
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

import matplotlib.pyplot as plt

plt.figure(figsize=(18, 6))
plt.scatter(all_labels, all_predictions)
plt.show();



INFO:root:Fitting Doc2VecVectorizer...
INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:collected 172981 word types and 5573 unique tags from a corpus of 5573 examples and 58171210 words
INFO:gensim.models.word2vec:Creating a fresh vocabulary
DEBUG:gensim.utils:starting a new internal lifecycle event log for Doc2Vec
INFO:gensim.utils:Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 65595 unique words (37.920349633774805%% of original 172981, drops 107386)', 'datetime': '2021-05-12T10:12:27.437297', 'gensim': '4.0.1', 'python': '3.9.4 (default, Apr  9 2021, 16:34:09) \n[GCC 7.3.0]', 'platform': 'Linux-5.8.0-50-generic-x86_64-with-glibc2.31', 'event': 'prepare_vocab'}
INFO:gensim.utils:Doc2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 58006051 word corpus (99.71608120236797%% of original 58171210, drops 165159)', 'datetime'

INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 40.88% examples, 1183137 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 43.82% examples, 1183566 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 46.78% examples, 1184696 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 49.72% examples, 1185312 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 52.63% examples, 1184746 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 55.59% examples, 1185167 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 58.57% examples, 1186154 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 61.53% examples, 1186783 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 64.44% examples, 1187445 words/s, in_qsize 15, out_qsize 0
INFO:gensi

INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 686 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 704 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH - 3 : training on 58171210 raw words (40856707 effective words) took 34.0s, 1200757 effective words/s
INFO:gensim.models.word2vec:EPOCH 4 - PROGRESS: at 2.87% examples, 1169664 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 4 - PROGRESS: at 5.78% examples, 1174453 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 4 - PROGRESS: at 8.76% examples, 1187870 words/s, in_qsize 16, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 4 - PROGRESS: at 11.86% examples, 1197183 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 4 - PRO

INFO:gensim.models.word2vec:EPOCH 5 - PROGRESS: at 77.48% examples, 1206712 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 5 - PROGRESS: at 80.51% examples, 1206556 words/s, in_qsize 16, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 5 - PROGRESS: at 83.46% examples, 1206983 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 5 - PROGRESS: at 86.43% examples, 1207428 words/s, in_qsize 16, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 5 - PROGRESS: at 89.38% examples, 1207569 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 5 - PROGRESS: at 92.21% examples, 1207788 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 5 - PROGRESS: at 95.14% examples, 1207769 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 5 - PROGRESS: at 98.10% examples, 1208323 words/s, in_qsize 15, out_qsize 0
DEBUG:gensim.models.word2vec:job loop exiting, total 5550 jobs
DEBUG:gensim.models.word2vec:worker exiting, processed 68

INFO:gensim.models.word2vec:EPOCH 7 - PROGRESS: at 29.84% examples, 1206533 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 7 - PROGRESS: at 32.68% examples, 1206169 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 7 - PROGRESS: at 35.73% examples, 1205793 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 7 - PROGRESS: at 38.81% examples, 1207379 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 7 - PROGRESS: at 41.84% examples, 1208773 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 7 - PROGRESS: at 44.82% examples, 1209065 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 7 - PROGRESS: at 47.86% examples, 1210066 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 7 - PROGRESS: at 50.82% examples, 1208801 words/s, in_qsize 14, out_qsize 2
INFO:gensim.models.word2vec:EPOCH 7 - PROGRESS: at 53.85% examples, 1209814 words/s, in_qsize 15, out_qsize 0
INFO:gensi

DEBUG:gensim.models.word2vec:worker exiting, processed 700 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 4 more threads
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 3 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 695 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 710 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads
DEBUG:gensim.models.word2vec:worker exiting, processed 697 jobs
INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.word2vec:EPOCH - 8 : training on 58171210 raw words (40858568 effective words) took 33.9s, 1204983 effective words/s
INFO:gensim.models.word2vec:EPOCH 9 - PROGRESS: at 2.87% examples, 1174030 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 9 - PROGRESS: at 5.81% examples, 11

INFO:gensim.models.word2vec:EPOCH 10 - PROGRESS: at 68.89% examples, 1209828 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 10 - PROGRESS: at 71.99% examples, 1211135 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 10 - PROGRESS: at 74.93% examples, 1211076 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 10 - PROGRESS: at 77.97% examples, 1211409 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 10 - PROGRESS: at 81.02% examples, 1211480 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 10 - PROGRESS: at 83.94% examples, 1211301 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 10 - PROGRESS: at 86.88% examples, 1211654 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 10 - PROGRESS: at 89.84% examples, 1212496 words/s, in_qsize 16, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 10 - PROGRESS: at 92.66% examples, 1212278 words/s, in_qsize 14, out_qsize 1
I

TypeError: read_labels() missing 1 required positional argument: 'lang'